In [1]:
from ling_features import non_answers, get_regexes_df
import pandas as pd
import nltk
from sklearn.metrics import confusion_matrix

In [2]:
regexes = get_regexes_df()
regexes

,regex_id,category,pattern
0,0,AFTERCALL,(?i)\b(off-line|offline|after the call|after c...
1,1,UNABLE,"(?i)\b(I|we)\b\s?([\w',&@#$%_\-\(\)\[\]]+\s){0..."
2,2,UNABLE,(?i)\b(I|we) have no idea\b
3,3,UNABLE,"(?i)\b(I|we) do(n('|’)t| not)\b\s?([\w',&@#$%_..."
4,4,UNABLE,"(?i)(n('|’)t|\bnot|cannot|without)\b\s?([\w',&..."
5,5,REFUSE,(?i)\b(address|addressed|addressing|explain|ex...
6,6,REFUSE,"(?i)\b(be\b\s?([\w',&@#$%_\-\(\)\[\]]+\s){0,1}..."
7,7,REFUSE,"(?i)(n('|’)t|\bnot|cannot|without)\b\s?([\w',&..."
8,8,REFUSE,"(?i)(n('|’)t|\bnot|cannot|without)\b\s?([\w',&..."
9,9,REFUSE,(?i)\b(difficult|impossible|infeasible|hard|de...


In [3]:
df = pd.read_csv("gold_standard.csv")

In [4]:
df

,file_name,section,answer_nums,answer_text,obs_type,is_unable,is_refuse,is_after_call,is_nonans
0,1000202_T,1,{26},"Well, I guess there was a couple of things to ...",train,True,True,False,True
1,1002829_T,1,"{124,126}",We are not sure that EPS will contract when it...,test,False,False,False,False
2,1003362_T,1,{29},"It will be a balance of both, Andy.",test,False,False,False,False
3,1006561_T,1,{24},"Well certainly, if you look at the December qu...",train,False,False,False,False
4,1008369_T,1,{64},I don't think it's as much as that but I'll ha...,train,True,False,False,True
...,...,...,...,...,...,...,...,...,...
1791,995280_T,1,{48},No. We do not see anything that would keep us ...,test,False,False,False,False
1792,995295_T,1,"{114,115,116}","I would say, yes, we have been more aggressive...",train,False,False,False,False
1793,997566_T,1,"{78,79}","Yeah, it’s pretty much holding across the boar...",train,False,False,False,False
1794,998196_T,1,"{24,26,28}",I think that when we originally did the transa...,train,False,False,False,False


In [5]:
df['non_answers'] = df['answer_text'].apply(nltk.sent_tokenize).map(non_answers)

In [6]:
def get_regex_ids(data):
    if data:
        regex_ids = [ eval(item)['regex_id'] for item in data]
        return [regexes['category'][i] for i in regex_ids]
    else:
        return None

In [7]:
df['non_answer_types'] = df['non_answers'].map(get_regex_ids)

In [8]:
def is_non_answer(types):
    if types:
        return set(['REFUSE', 'UNABLE', 'AFTERCALL']).intersection(types) is not None
    else:
        return False

In [9]:
df['is_nonans_calc'] =  df['non_answer_types'].map(is_non_answer)

In [10]:
def print_stats(df):
    confusion = confusion_matrix(df['is_nonans'], df['is_nonans_calc'])
    tn, fp, fn, tp = confusion.ravel()
    print("Accuracy {:.2f}%".format( 100 * (tp + tn)/(tp + tn + fp + fn)))
    print("Precision {:.2f}%".format( 100 * tp/(tp + fp)))
    print("True positive rate {:.2f}%".format( 100 * tp/(tp + fn)))

In [11]:
print_stats(df[df['obs_type']=='test'])

Accuracy 89.20%
Precision 58.95%
True positive rate 78.87%


In [12]:
print_stats(df[df['obs_type']=='train'])

Accuracy 91.05%
Precision 68.38%
True positive rate 82.78%
